In [1]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('../Train.csv')
test = pd.read_csv('../Test.csv')

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression , RidgeClassifier, Huber
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier , ExtraTreesClassifier , AdaBoostClassifier
from sklearn.model_selection import KFold ,StratifiedKFold
from sklearn.metrics import log_loss
import warnings 
warnings.filterwarnings('ignore')

In [5]:
train = train[train['time']<2*1e6]
train = train[train['time']!=1804691]
train = train[train['Store_Ratio']>0]
# train.shape
train.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
0,0.407,0.00380,2,-35.865,1,0.9920,0.944,0.0988,0.1100,113.911,189125,4
1,0.234,0.10500,0,-19.884,1,0.9440,0.900,0.1290,0.0382,76.332,186513,4
2,0.668,0.72600,9,-7.512,1,0.0387,0.000,0.1340,0.4530,124.075,172143,4
3,0.184,0.00561,4,-34.357,1,0.8490,0.931,0.1110,0.0641,79.037,184000,5
4,0.231,0.13100,6,-22.842,1,0.9370,0.000,0.1090,0.0677,109.560,186507,3


In [6]:
train.shape

(18189, 12)

In [7]:
train.popularity.value_counts()

4    15229
3     2165
5      472
1      323
Name: popularity, dtype: int64

In [8]:
df = train.append(test)

In [9]:
df.columns

Index(['Store_Ratio', 'Basket_Ratio', 'Category_1', 'Store_Score',
       'Category_2', 'Store_Presence', 'Score_1', 'Score_2', 'Score_3',
       'Score_4', 'time', 'popularity'],
      dtype='object')

In [53]:
# dummy = test[test['Store_Presence']==0]
# dummy.head()

In [54]:
# for col in df.columns:
#     if 0 in df[col].values:
#         print(col)

In [10]:
# df['time']  = df['time']/3600
# df['Score_2'] = np.log1p(df['Score_2'])
df['Store_Pre_Bin'] = df['Store_Presence'].apply(lambda x: 1 if x<=0.5 else 0)
df['avg_score'] = (df['Score_1']+df['Score_2']+df['Score_3']+df['Score_4'])/4 

In [56]:
# df['Basket_Ratio_sum_Store_Ratio'] = df['Basket_Ratio'] + df['Store_Ratio']
# df['Score_3_ratio_Store_Ratio'] = df['Score_3']/df['Store_Ratio']
# df['Score_3_multiply_Store_Ratio'] = df['Score_3']*df['Store_Ratio']
# df['Score_3_ratio_Store_Ratio'] = df['Score_3']/df['Store_Ratio']
# df['Store_Ratio_diff_Store_Presence'] = df['Store_Ratio']-df['Store_Presence']


In [11]:
df.fillna(0)

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity,Store_Pre_Bin,avg_score
0,0.407,0.00380,2,-35.865,1,0.99200,0.944000,0.0988,0.1100,113.911,189125,4.0,0,28.765950
1,0.234,0.10500,0,-19.884,1,0.94400,0.900000,0.1290,0.0382,76.332,186513,4.0,0,19.349800
2,0.668,0.72600,9,-7.512,1,0.03870,0.000000,0.1340,0.4530,124.075,172143,4.0,1,31.165500
3,0.184,0.00561,4,-34.357,1,0.84900,0.931000,0.1110,0.0641,79.037,184000,5.0,0,20.035775
4,0.231,0.13100,6,-22.842,1,0.93700,0.000000,0.1090,0.0677,109.560,186507,3.0,0,27.434175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12135,0.171,0.11800,2,-25.360,1,0.86100,0.871000,0.0932,0.0539,78.303,192039,0.0,0,19.830275
12136,0.744,0.85900,11,-3.166,1,0.02560,0.000063,0.0881,0.7870,91.977,232787,0.0,1,23.213041
12137,0.572,0.85200,6,-4.712,0,0.00105,0.000000,0.0844,0.3780,129.969,216411,0.0,1,32.607850
12138,0.761,0.52500,11,-6.900,1,0.44000,0.000007,0.0921,0.5310,80.870,238805,0.0,1,20.373277


In [58]:
for col in df.columns:
    print(col,np.max(df[col]))

Store_Ratio 0.998
Basket_Ratio 1.0
Category_1 11
Store_Score 0.662
Category_2 1
Store_Presence 0.996
Score_1 1.0
Score_2 0.6901426715396466
Score_3 1.0
Score_4 219.701
time 495.0494444444444
popularity 5.0
Store_Pre_Bin 1
avg_score 55.36519037002033
score_by_basket 172726.82082312694
Basket_Ratio_sum_Store_Ratio 1.882
Score_3_ratio_Store_Ratio 7.779578606158833
Score_3_multiply_Store_Ratio 0.9321879999999999
Store_Ratio_diff_Store_Presence 0.97691


In [59]:
train,test = df.head(len(train)),df.tail(len(test))
test.drop(columns=['popularity'],axis=1,inplace=True)

In [60]:
X = train.drop('popularity',axis=1)
Y = train['popularity']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_ = scaler.fit_transform(X)
X = pd.DataFrame(X_,columns=X.columns)

In [ ]:

0.40622963042251287
0.4147608064892967
0.4083242484803806
0.41242385360564
0.3933343406766553
0.3833319521429497
0.3776756966009344
0.40536682578445876
0.38166896790431204
0.3952124277656655
Mean OOF Score : 0.3978328749872806

In [61]:
folds = StratifiedKFold(n_splits=10,shuffle=True ,random_state=42)

oof_preds = []
oof_scores = []

for i , (train_idx,test_idx) in enumerate(folds.split(X,Y)):
  train_set = (X.iloc[train_idx],Y.iloc[train_idx])
  test_set = (X.iloc[test_idx],Y.iloc[test_idx])

  model = CatBoostClassifier(loss_function='MultiClass', 
                         eval_metric='TotalF1', 
                         depth=7,
                         verbose=0
)
  model.fit(*train_set)
  preds = model.predict_proba(test_set[0])
  score = log_loss(test_set[1],preds)
  
  oof_pred_fold = model.predict_proba(scaler.transform(test))
  oof_preds.append(oof_pred_fold)

  print(score)
  oof_scores.append(score)

print("Mean OOF Score :",np.mean(oof_scores))
final_preds = np.mean(np.array(oof_preds),axis=0)

0.4077571456881573
0.410899445614295
0.41174113748078867
0.4161660937967123
0.3944097508674703
0.38319656503468696
0.3797521124535059
0.4004541570696445
0.3843497714203671
0.39513380684251465
Mean OOF Score : 0.39838599862681423


In [62]:
temp = pd.DataFrame(final_preds)
temp.head()


,0,1,2,3
0,0.017498,0.768680,0.162643,0.051180
1,0.004417,0.022197,0.971798,0.001589
2,0.000561,0.024412,0.965735,0.009292
3,0.002288,0.036719,0.951077,0.009916
4,0.000128,0.001720,0.996169,0.001984


In [63]:
temp.columns

RangeIndex(start=0, stop=4, step=1)

In [64]:
sub = pd.DataFrame()
sub['0'] = 0
sub['1'] = temp[0]
sub['2'] = temp[1]
sub['3'] = temp[2]
sub['4'] = temp[3]
sub = sub.fillna(0)

In [65]:
sub.head()

,0,1,2,3,4
0,0.0,0.017498,0.768680,0.162643,0.051180
1,0.0,0.004417,0.022197,0.971798,0.001589
2,0.0,0.000561,0.024412,0.965735,0.009292
3,0.0,0.002288,0.036719,0.951077,0.009916
4,0.0,0.000128,0.001720,0.996169,0.001984


In [66]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [67]:
#there are duplicates between test and train, for those rows we can directly use train target values
#storing the target values of common rows in test and train
temp1=''
for i in test.columns:
    temp1=temp1+i+train[i].astype(str)
train['ALL']=temp1
temp2=''
for i in test.columns:
    temp2=temp2+i+test[i].astype(str)
test['ALL']=temp2
encoding=train.groupby('ALL')['popularity'].median()
y_from_train=test['ALL'].map(encoding)
temp = pd.DataFrame()
temp['pop'] = y_from_train
temp['pop'] = temp['pop'].fillna(-999)
x = temp[temp['pop']!=-999]
for i in x.index.values:
#     print(x.iloc[i].values[0])
    try:
        if x.loc[i].values[0] ==0:
            sub.loc[i] =[1.0,0.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==1:
            sub.loc[i] =[0.0,1.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==3:
            sub.loc[i] =[0.0,0.0,1.0,0.0,0.0]
        elif x.loc[i].values[0] ==4:
            sub.loc[i] =[0.0,0.0,0.0,1.0,0.0]
        elif x.loc[i].values[0] ==5:
            sub.loc[i] =[0.0,0.0,0.0,0.0,1.0]
        else:
            print('.',end='')
    except:
        print(i)
        print(x.loc[i].values[0])

In [68]:
sub.head()

,0,1,2,3,4
0,0.0,0.000000,1.000000,0.000000,0.000000
1,0.0,0.004417,0.022197,0.971798,0.001589
2,0.0,0.000561,0.024412,0.965735,0.009292
3,0.0,0.002288,0.036719,0.951077,0.009916
4,0.0,0.000000,0.000000,1.000000,0.000000


In [70]:
dummy = test[test['Store_Ratio']==0]
dummy.index
for i in dummy.index.values:
    sub.iloc[i] =[1.0,0.0,0.0,0.0,0.0]
sub.head()

,0,1,2,3,4
0,0.0,0.000000,1.000000,0.000000,0.000000
1,0.0,0.004417,0.022197,0.971798,0.001589
2,0.0,0.000561,0.024412,0.965735,0.009292
3,0.0,0.002288,0.036719,0.951077,0.009916
4,0.0,0.000000,0.000000,1.000000,0.000000


In [71]:
np.sum(sub['0'])

11.0

In [72]:
sub.to_csv('submission_2801_v1.csv',index=False)

In [73]:
from sklearn.model_selection import train_test_split

In [75]:
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify = Y,test_size=0.1)

In [76]:
# !pip install optuna
import optuna.integration.lightgbm as lgb
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_test, label=y_test)

params = {
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
}

model = lgb.train(
        params, dtrain, valid_sets=[dtrain, dval], verbose_eval=100, early_stopping_rounds=100
    )

prediction = np.rint(model.predict(X_test, num_iteration=model.best_iteration))
accuracy = accuracy_score(y_test, prediction)

best_params = model.params
print("Best params:", best_params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in best_params.items():
  print("    {}: {}".format(key, value))


[I 2021-01-28 19:44:03,293] A new study created in memory with name: no-name-4d9361f4-58b1-4a3d-ab79-c3796833ab2b
feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1


feature_fraction, val_score: 1.000000:  14%|#4        | 1/7 [00:00<00:04,  1.32it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1


feature_fraction, val_score: 1.000000:  29%|##8       | 2/7 [00:01<00:03,  1.33it/s]

...

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1


min_data_in_leaf, val_score: 1.000000:  80%|########  | 4/5 [00:03<00:00,  1.11it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1


min_data_in_leaf, val_score: 1.000000: 100%|##########| 5/5 [00:04<00:00,  1.11it/s]


Best params: {'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 31, 'feature_fraction': 0.8, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100}
  Accuracy = 0.8372732270478285
  Params: 
    metric: auc
    verbosity: -1
    boosting_type: gbdt
    feature_pre_filter: False
    lambda_l1: 0.0
    lambda_l2: 0.0
    num_leaves: 31
    feature_fraction: 0.8
    bagging_fraction: 1.0
    bagging_freq: 0
    min_child_samples: 20
    num_iterations: 1000
    early_stopping_round: 100


In [90]:
params = {
    'metric': 'multi_logloss',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'feature_pre_filter': False,
    'lambda_l1': 0.0,
    'lambda_l2': 0.0,
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'min_child_samples': 20,
    'num_iterations': 1000
}

In [91]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(**params)

In [92]:
model.fit(X_train,y_train)

LGBMClassifier(bagging_fraction=1.0, bagging_freq=0, feature_fraction=0.8,
               feature_pre_filter=False, lambda_l1=0.0, lambda_l2=0.0,
               metric='multi_logloss', num_iterations=1000, verbosity=-1)

In [57]:
preds_val = model.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,preds_val))

0.8565145684442002


In [93]:
final_preds = model.predict_proba(test.drop(columns=['ALL'],axis=1))
final_preds

ValueError: Number of features of the model must match the input. Model n_features_ is 18 and input n_features is 11 

In [ ]:
temp = pd.DataFrame(final_preds)
sub = pd.DataFrame()
sub['0'] = 0
sub['1'] = temp[0]
sub['2'] = temp[1]
sub['3'] = temp[2]
sub['4'] = temp[3]


In [ ]:
sub = sub.fillna(0)

In [62]:
#there are duplicates between test and train, for those rows we can directly use train target values
#storing the target values of common rows in test and train
temp1=''
for i in test.columns:
    temp1=temp1+i+train[i].astype(str)
train['ALL']=temp1
temp2=''
for i in test.columns:
    temp2=temp2+i+test[i].astype(str)
test['ALL']=temp2
encoding=train.groupby('ALL')['popularity'].median()
y_from_train=test['ALL'].map(encoding)
temp = pd.DataFrame()
temp['pop'] = y_from_train
temp['pop'] = temp['pop'].fillna(-999)
x = temp[temp['pop']!=-999]
for i in x.index.values:
#     print(x.iloc[i].values[0])
    try:
        if x.loc[i].values[0] ==0:
            sub.loc[i] =[1.0,0.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==1:
            sub.loc[i] =[0.0,1.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==3:
            sub.loc[i] =[0.0,0.0,1.0,0.0,0.0]
        elif x.loc[i].values[0] ==4:
            sub.loc[i] =[0.0,0.0,0.0,1.0,0.0]
        elif x.loc[i].values[0] ==5:
            sub.loc[i] =[0.0,0.0,0.0,0.0,1.0]
        else:
            print('.',end='')
    except:
        print(i)
        print(x.loc[i].values[0])

In [63]:
dummy = test[test['Store_Ratio']==0]
dummy.index
for i in dummy.index.values:
    sub.iloc[i] =[1.0,0.0,0.0,0.0,0.0]
sub.head()

,0,1,2,3,4
0,0.0,0.000000,1.000000,0.000000,0.000000
1,0.0,0.004859,0.196113,0.787081,0.011947
2,0.0,0.019107,0.177063,0.788720,0.015110
3,0.0,0.007202,0.188808,0.794248,0.009742
4,0.0,0.000000,0.000000,1.000000,0.000000


In [64]:
np.sum(sub['0'])

11.0

In [65]:
sub.to_csv('sub_2501_v2_lgb.csv',index=False)